In [60]:
from datasets import load_from_disk
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm
import evaluate
import random
import argparse
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import pos_tag

In [61]:
def custom_transform(example):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.
    
    # You should update example["text"] using your transformation
        
    raise NotImplementedError
    
    
    ##### YOUR CODE ENDS HERE ######
    
    return example




In [62]:
# Define the directory where the dataset is saved
load_directory = "dataset/"

# Load the dataset from the specified directory
loaded_data = load_from_disk(load_directory)

# Now you can access the individual splits (train, test, unsupervised) as follows:
train_dataset = loaded_data["train"]
test_dataset = loaded_data["test"]
unsupervised_dataset = loaded_data["unsupervised"]

In [63]:
# replacement probability
probability = 0.6

# Generate a random number between 0 and 1
random_number = random.random()

In [64]:
example = test_dataset[0]['text']

In [107]:
def get_wordnet_pos(treebank_tag): # Convert POS-Tag from treebank_tag to Wordnet tags
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None  # If no match is found, return None
    


In [163]:
def replace_with_synonym(text: str, pct_synonyms: float) -> str:
    """Replace {pct_synonyms}% of words in a sentence with their synonyms
    :param text (str):
    :param pct_synonyms (float): percentage of words to be replaced. Between 0 and 1
    :return: 
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)
    n_tokens = len(word_list)
    n_synonyms = int(n_tokens*pct_synonyms)

    # Perform POS tagging to help selecting synonym
    pos_tags = pos_tag(word_list)
    syn_dict = {}
    for i in range(n_synonyms):
        chosen_word = random.choice(word_list)
        synonyms = wn.synsets(chosen_word)

        # If chosen word doesn't have a synonym
        # retry until finding a word that does.
        max_retry = len(word_list)
        retry = 0 
        while (len(synonyms) == 0 and retry <= max_retry):
            chosen_word = random.choice(word_list)
            synonyms = wn.synsets(chosen_word)

            if len(synonyms) > 0:
                # Print the word and its POS tag
                chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]
                chosen_word_wordnet_tag = get_wordnet_pos(chosen_word_tag)
                
                selected_synonym = chosen_word
                for syn in synonyms:
                    # Only consider synonyms that have the same POS_TAG
                    if syn.pos() == chosen_word_wordnet_tag:
                        for lemma in syn.lemmas():
                            if (lemma.name() != chosen_word):
                                # Select the first synonym that appears
                                selected_synonym = lemma.name().replace('_', ' ') # Replace underscores for multi-word synonyms
                                break
                        break
                    break
                if selected_synonym == chosen_word:
                    synonyms = [] # continue while loop
                else:
                    # print("selected_synonym: ", selected_synonym)
                    syn_dict[chosen_word] = selected_synonym

    # return syn_dict
    # Replace selected words
    for old, new in syn_dict.items():
        text = text.replace(old, new)
    return text, syn_dict

In [164]:
N = 5
for i, data in enumerate(test_dataset):
    sentence = data["text"]
    print("Original\n", sentence)
    sentence_new, syn_dict = replace_with_synonym(sentence, 0.40)
    print("Synonym\n", sentence_new)
    print(syn_dict)
    if i > N:
        break

Original
 I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they hav

In [65]:
word_list  = word_tokenize(example)

In [67]:
chosen_word = random.choice(word_list)
synonyms = wn.synsets(chosen_word)

# If chosen word doesn't have a synonym select retry until finding a word that does.
max_retry = len(word_list)
retry = 0 
while (len(synonyms) == 0 and retry <= max_retry):
    chosen_word = random.choice(word_list)
    synonyms = wn.synsets(chosen_word)

In [74]:
# Perform POS tagging
pos_tags = pos_tag(word_list)

# Print the word and its POS tag
chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]

In [79]:
synonym_list = set()
for syn in synonyms:
    # Only consider synonyms that have the same POS_TAG
    if syn.pos() == chosen_word_wordnet_tag:
        for lemma in syn.lemmas():
            if (lemma.name() != chosen_word) & (lemma):

                synonym_list.add(lemma.name().replace('_', ' ')) # Replace underscores for multi-word synonyms